<a href="https://colab.research.google.com/github/CiaraFarrellSETU/Dissertation_code/blob/main/vgg16_6classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from timeit import default_timer as timer

start = timer()

In [ ]:
import os
import cv2
from PIL import Image
import tensorflow as tf
from keras import backend as K
from keras.models import load_model
from keras.preprocessing.image import img_to_array
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import zipfile

with zipfile.ZipFile('/content/gdrive/MyDrive/dogeye.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/unzipp_d')  # Replace with desired destination directory

In [ ]:
base_dir = '/content/unzipp_d/dogeye'

train_dir = '/content/unzipp_d/dogeye/train'
cat_train_dir = '/content/unzipp_d/dogeye/train/cataract'
healthy_train_dir = '/content/unzipp_d/dogeye/train/healthy'
mas_train_dir = '/content/unzipp_d/dogeye/train/mas'
conj_train_dir = '/content/unzipp_d/dogeye/train/conj'
ulcer_train_dir = '/content/unzipp_d/dogeye/train/ulcer'
nuc_train_dir = '/content/unzipp_d/dogeye/train/nuc'

test_dir = '/content/unzipp_d/dogeye/test'
cat_test_dir = '/content/unzipp_d/dogeye/test/cataract'
healthy_test_dir = '/content/unzipp_d/dogeye/test/healthy'
mas_test_dir = '/content/unzipp_d/dogeye/test/mas'
conj_test_dir = '/content/unzipp_d/dogeye/test/conj'
ulcer_test_dir = '/content/unzipp_d/dogeye/test/ulcer'
nuc_test_dir = '/content/unzipp_d/dogeye/test/nuc'

val_dir = '/content/unzipp_d/dogeye/valid'
cat_val_dir = '/content/unzipp_d/dogeye/valid/cataract'
healthy_val_dir = '/content/unzipp_d/dogeye/valid/healthy'
mas_val_dir = '/content/unzipp_d/dogeye/valid/mas'
conj_val_dir = '/content/unzipp_d/dogeye/valid/conj'
ulcer_val_dir = '/content/unzipp_d/dogeye/valid/ucler'
nuc_val_dir = '/content/unzipp_d/dogeye/valid/nuc'

In [ ]:
import os
#train
num_cat_train = len(os.listdir(cat_train_dir))
num_healthy_train = len(os.listdir(healthy_train_dir))

num_mas_train = len(os.listdir(mas_train_dir))
num_conj_train = len(os.listdir(conj_train_dir))
num_ulcer_train = len(os.listdir(ulcer_train_dir))
num_nuc_train = len(os.listdir(nuc_train_dir))

#test
num_cat_test = len(os.listdir(cat_test_dir))
num_healthy_test = len(os.listdir(healthy_test_dir))

num_mas_test = len(os.listdir(mas_test_dir))
num_conj_test = len(os.listdir(conj_test_dir))
num_ulcer_test = len(os.listdir(ulcer_test_dir))
num_nuc_test = len(os.listdir(nuc_test_dir))

#val
num_cat_val = len(os.listdir(cat_val_dir))
num_healthy_val = len(os.listdir(healthy_val_dir))

num_mas_val = len(os.listdir(mas_val_dir))
num_conj_val = len(os.listdir(conj_val_dir))
num_ulcer_val = len(os.listdir(ulcer_val_dir))
num_nuc_val = len(os.listdir(nuc_val_dir))

In [ ]:
print("--")
print("Total Training cat Images",num_cat_train)
print("Total Training conj Images",num_conj_train)

print("Total Training healthy Images",num_healthy_train)
print("Total Training mas Images",num_mas_train)
print("Total Training nuc Images",num_nuc_train)
print("Total Training ulcer Images",num_ulcer_train)
print("--")
print("Total test cat Images",num_cat_test)
print("Total test healthy Images",num_healthy_test)

print("Total test mas Images",num_mas_test)
print("Total Training conj Images",num_conj_test)
print("Total Training ulcer Images",num_ulcer_test)
print("Total Training nuc Images",num_nuc_test)
print("--")
print("Total validation cat Images",num_cat_val)
print("Total validation healthy Images",num_healthy_val)

print("Total validation mas Images",num_mas_val)
print("Total Training conj Images",num_conj_val)
print("Total Training ulcer Images",num_ulcer_val)
print("Total Training nuc Images",num_nuc_val)
print("--")

In [ ]:
total_train = num_cat_train+num_healthy_train+num_mas_train+num_conj_train+num_ulcer_train+num_nuc_train
total_validation = num_cat_val+num_healthy_val+num_mas_val+num_conj_val+num_ulcer_val+num_nuc_val
total_test = num_cat_test+num_healthy_test+num_mas_test+num_conj_test+num_ulcer_test+num_nuc_test

total_images = total_train + total_validation + total_test

print("--")
print("Total Training Images",total_train)
print("--")
print("Total Validation Images",total_validation)
print("--")
print("Total Testing Images",total_test)
print("--")
print("Total Images",total_images)
print("--")

In [ ]:
IMG_SHAPE  = 224
batch_size = 32 #64 as 32 too slow

In [ ]:
image_gen_train = ImageDataGenerator(rescale = 1./255)
train_data_gen = image_gen_train.flow_from_directory(batch_size = batch_size,
directory = train_dir,
shuffle= True,
target_size = (IMG_SHAPE,IMG_SHAPE),
class_mode = 'categorical')
image_generator_validation = ImageDataGenerator(rescale=1./255)
val_data_gen = image_generator_validation.flow_from_directory(batch_size=batch_size,
directory=val_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='categorical')
image_gen_test = ImageDataGenerator(rescale=1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=batch_size,
directory=test_dir,
target_size=(IMG_SHAPE, IMG_SHAPE),
class_mode='categorical')

In [ ]:
os.listdir(train_dir)

In [ ]:
print(train_data_gen[0][0].shape, train_data_gen[0][1].shape)

In [ ]:
print(train_data_gen[0][1])  #checking labels

In [ ]:
pre_trained_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

In [ ]:
for layer in pre_trained_model.layers:
  print(layer.name)
layer.trainable = False

In [ ]:
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
x = tf.keras.layers.GlobalMaxPooling2D()(last_output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(6, activation='softmax')(x)

In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)


In [ ]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy']) #only for two classes
#model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # didn't work
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # Adjust learning rate as needed

vgg_classifier = model.fit(train_data_gen,
                           steps_per_epoch=(total_train // batch_size),
                           epochs=25,
                           validation_data=val_data_gen,
                           validation_steps=(total_validation // batch_size),
                           batch_size=batch_size,
                           verbose=1)

In [ ]:
#model fit with early stopping

# from tensorflow.keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# vgg_classifier = model.fit(train_data_gen,
#                           steps_per_epoch=(total_train // batch_size),
#                           epochs=25,
#                           validation_data=val_data_gen,
#                           validation_steps=(total_validation // batch_size),
#                           batch_size=batch_size,
#                           verbose=1,
#                           callbacks=[early_stopping])


In [ ]:
result = model.evaluate(test_data_gen,batch_size=batch_size)
print("test_loss, test accuracy",result)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

true_labels = []
predicted_probabilities = []

for _ in range(len(test_data_gen)):
    x, y = test_data_gen.next()
    true_labels.extend(np.argmax(y, axis=1))
    predicted_probabilities.extend(model.predict(x))

predicted_classes = np.argmax(predicted_probabilities, axis=1)

print("Classification Report:\n")
print(classification_report(true_labels, predicted_classes))


In [ ]:
from sklearn.metrics import confusion_matrix
_, test_acc = result

predictions = model.predict(test_data_gen, batch_size=batch_size)

true_labels = test_data_gen.classes

import numpy as np
predicted_classes = np.argmax(predictions, axis=1)
cm = confusion_matrix(true_labels, predicted_classes)
print("Confusion Matrix:\n")
print(cm)
print("\nTest Accuracy:", test_acc)

In [ ]:
end = timer()
print(end - start)